# <font color='orange'> Import library </font>

In [ ]:
!pip install -q transformers

In [ ]:
!pip install -q datasets

In [ ]:
!pip install -q albumentations

In [ ]:
!pip install -q evaluate

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time

import torch
from torch import nn
import evaluate

# <font color='orange'> Read data </font>

In [ ]:
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

class ImageSegmentationDataset(Dataset):
  """Image segmentation dataset."""

  def __init__(self, dataset, data_path, transform):
    """
    Args:
        dataset
    """
    self.dataset = dataset
    self.data_path = data_path
    self.transform = transform

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    img_path = os.path.join(self.data_path, self.dataset.iloc[idx, 0])
    label_path = os.path.join(self.data_path, self.dataset.iloc[idx, 1])

    original_image = Image.open(img_path).convert('RGB')
    original_segmentation_map = Image.open(label_path)

    original_image = np.array(original_image)
    original_segmentation_map = np.array(original_segmentation_map.convert('1'))*1

    transformed = self.transform(image=original_image, mask=original_segmentation_map)
    image, segmentation_map = transformed['image'], transformed['mask']

    # convert to C, H, W
    image = image.transpose(2, 0, 1)

    return image, segmentation_map, original_image, original_segmentation_map

In [ ]:
def train_val_split(dataset, validation_split=0.2):
    """make dataloader for pytorch training

    Args:
        dataset ([object]): [the dataset object]
        train_batch_size ([int]): [training batch size]
        validation_split (float, optional): [validation ratio]. Defaults to 0.2.

    Returns:
        [type]: [description]
    """
    # number of samples in train and test set
    train_len = int(len(dataset) * (1 - validation_split))
    test_len = len(dataset) - train_len
    train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])
    # create train_loader
    #train_loader = torch.utils.data.DataLoader(train_set, batch_size=train_batch_size, shuffle=True,)
    # create test_loader
    #test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False,)
    return train_set, test_set

In [ ]:
import albumentations as A

ADE_MEAN = np.array([123.675, 116.280, 103.530]) / 255
ADE_STD = np.array([58.395, 57.120, 57.375]) / 255

train_transform = A.Compose([
    A.RandomCrop(width=224, height=224),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),
])

test_transform = A.Compose([
    A.Resize(width=224, height=224),
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),

])

In [ ]:
datasets = []

for i in range(0,5):
    df_train = pd.read_csv(f'/kaggle/input/ebhi-seg-csv/class/Polyp/train_fold_{i}.csv')
    df_test = pd.read_csv(f'/kaggle/input/ebhi-seg-csv/class/Polyp/test_fold_{i}.csv')

    ## train test split
    dataset = ImageSegmentationDataset(df_train, data_path='/kaggle/input/ebhi-seg/', transform=train_transform)
    train_dataset, valid_dataset = train_val_split(dataset, validation_split=0.5)
    test_dataset = ImageSegmentationDataset(df_test, data_path='/kaggle/input/ebhi-seg/', transform=test_transform)
    
    datasets.append((train_dataset, valid_dataset, test_dataset))

In [ ]:
#id2label = {0:'negative', 1:'positive'}
#labels = [id2label[label] for label in np.unique(segmentation_map)]
#print(labels)

# <font color='orange'> Create PyTorch DataLoaders </font>

In [ ]:
from transformers import Mask2FormerImageProcessor

# Create a preprocessor
preprocessor = Mask2FormerImageProcessor(
    ignore_index=-1, reduce_labels=False, do_resize=False,
    do_rescale=False, do_normalize=False,
    num_classes=2,
)

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    inputs = list(zip(*batch))
    images = inputs[0]
    segmentation_maps = inputs[1]
    # this function pads the inputs to the same size,
    # and creates a pixel mask
    # actually padding isn't required here since we are cropping
    batch = preprocessor(
        images,
        segmentation_maps=segmentation_maps,
        return_tensors="pt",
    )

    batch["original_images"] = inputs[2]
    batch["original_segmentation_maps"] = inputs[3]

    return batch

In [ ]:
dataloaders = []
for i, (train_set, valid_set, test_set) in enumerate(datasets):
    train_dataloader = DataLoader(train_set, batch_size=8, shuffle=True, collate_fn=collate_fn)
    val_dataloader = DataLoader(valid_set, batch_size=4, shuffle=True, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_set, batch_size=4, shuffle=False, collate_fn=collate_fn)
    
    dataloaders.append((train_dataloader, val_dataloader, test_dataloader))

# <font color='orange'> Verify data (!!) </font>

Next, it's ALWAYS very important to check whether the data you feed to the model actually makes sense. It's one of the main principles of [this amazing blog post](http://karpathy.github.io/2019/04/25/recipe/), if you wanna debug your neural networks.

Let's check the first batch, and its content.

In [ ]:
import torch
train_dataloader = dataloaders[0][1]

batch = next(iter(train_dataloader))
for k,v in batch.items():
    print(type(v))
    if isinstance(v, torch.Tensor):
        print(k,v.shape)
    else:
        print(k,v[0].shape)

# <font color='orange'>  Define model </font>

Next, we define the model. We equip the model with pretrained weights from the 🤗 hub. We will replace only the classification head. For that we provide the id2label mapping, and specify to ignore mismatches keys to replace the already fine-tuned classification head.

In [ ]:
from transformers import Mask2FormerForUniversalSegmentation
from torch import Tensor

class PolypSegmentationNetwork(nn.Module):
    def __init__(self):
        super(PolypSegmentationNetwork, self).__init__()
        
        # Tạo instance của Mask2FormerForSegmentation
        self.mask2former = Mask2FormerForUniversalSegmentation.from_pretrained(
            "facebook/mask2former-swin-base-coco-panoptic")
        
    
    def forward(self, pixel_values, class_labels=None, mask_labels=None):
        outputs = self.mask2former(
            pixel_values=pixel_values, 
            class_labels=class_labels, 
            mask_labels=mask_labels
        )
        
        # Tính toán đầu ra của lớp linear cuối cùng
        #logits = self.segmentation_head(outputs.last_hidden_state)
        return outputs

In [ ]:
#from transformers import Mask2FormerConfig

#config = Mask2FormerConfig(feature_size=224, mask_feature_size=224)
#config.save_pretrained('./config')

#model = PolypSegmentationNetwork()

# <font color='orange'> Train the model </font>

In [ ]:
import evaluate

id2label = {0:'background',1:'polyp'}
metric = evaluate.load("mean_iou")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from transformers import Mask2FormerForUniversalSegmentation

#model = Mask2FormerForUniversalSegmentation.from_pretrained(
#    "facebook/mask2former-swin-base-coco-panoptic").to(device)

model = PolypSegmentationNetwork()
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2, factor=0.5)

In [ ]:
import torch
from tqdm.auto import tqdm

def train_one_fold(
    model, 
    train_dataloader, 
    val_dataloader,
    id2label,
    device, 
    optimizer,
    #scheduler=None,
    metric,
    idx_fold=0,
    epochs=20
):
    running_loss = 0.0
    num_samples = 0
    best_iou = 0.0
    exec_times = []
    
    for epoch in range(epochs):
        print("Epoch:", epoch)
        start_time = time.time()
        
        model.train()
        for idx, batch in enumerate(tqdm(train_dataloader)):
            # Reset the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(
                pixel_values=batch["pixel_values"].to(device),
                class_labels=[labels.to(device) for labels in batch["class_labels"]],
                mask_labels=[labels.to(device) for labels in batch["mask_labels"]],
            )

            # Backward propagation
            loss = outputs.loss
            loss.backward()

            batch_size = batch["pixel_values"].size(0)
            running_loss += loss.item()
            num_samples += batch_size

            if idx % 100 == 0:
                print("Loss:", running_loss/num_samples)

            # Optimization
            optimizer.step()

        model.eval()
        for idx, batch in enumerate(tqdm(val_dataloader)):
            #if idx > 8:
            #   break
            
            # Forward pass
            with torch.no_grad():
                outputs = model(pixel_values=batch["pixel_values"].to(device))

            # get original images
            original_images = batch["original_images"]
            target_sizes = [(image.shape[0], image.shape[1]) for image in original_images]
            # predict segmentation maps
            predicted_segmentation_maps = preprocessor.post_process_semantic_segmentation(outputs, target_sizes=target_sizes)

            # get ground truth segmentation maps
            ground_truth_segmentation_maps = batch["original_segmentation_maps"]

            metric.add_batch(references=ground_truth_segmentation_maps, predictions=predicted_segmentation_maps)

        # NOTE this metric outputs a dict that also includes the mIoU per category as keys
        # so if you're interested, feel free to print them as well
        iou = metric.compute(num_labels = len(id2label), ignore_index = -1)['mean_iou']
        
        if(best_iou < iou):
            best_iou = iou
            print(f"Mean IoU = " + str(iou) + " ===> Save best epoch")
            torch.save(model.state_dict(), "./fold" + str(idx_fold) +  "_model.pt")
        else:
            print("Mean IoU = " + str(iou) + " ===> No saving")
        
        exec_time = time.time()-start_time
        exec_times.append(exec_time)
        print("Take %f seconds to train epoch %d." %(exec_time, epoch))
    
    df_execute = pd.DataFrame(data={
        "time": exec_times
    })
    df_execute.to_csv("exec_times.csv", index=None)
    
    return best_iou

In [ ]:
def test_result(model, test_dataloader, id2label, device):
    #metric = evaluate.load("mean_iou")
    model.eval()
    
    pred_segmentation_maps = []
    gnd_truth_segmentation_maps = []
    
    for idx, batch in enumerate(tqdm(test_dataloader)):

        pixel_values = batch["pixel_values"]

        # Forward pass
        with torch.no_grad():
            outputs = model(pixel_values=pixel_values.to(device))

        # get original images
        original_images = batch["original_images"]
        target_sizes = [(image.shape[0], image.shape[1]) for image in original_images]
        
        # predict segmentation maps
        batch_predict_maps = preprocessor.post_process_semantic_segmentation(
            outputs, target_sizes=target_sizes)
        pred_segmentation_maps.extend(batch_predict_maps)
        
        # get ground truth segmentation maps
        batch_ground_truth_maps = batch["original_segmentation_maps"]
        gnd_truth_segmentation_maps.extend(batch_ground_truth_maps)
        
        metric.add_batch(references=batch_ground_truth_maps, predictions=batch_predict_maps)
    
    #iou = metric.compute(ignore_index=-1, num_labels = len(id2label))['mean_iou']
    #print("Mean IoU:", iou)
    return (gnd_truth_segmentation_maps, pred_segmentation_maps)
    
# NOTE this metric outputs a dict that also includes the mIoU per category as keys
# so if you're interested, feel free to print them as well
# print("Mean IoU:", metric.compute(ignore_index=-1,num_labels = len(id2label))['mean_iou'])

# <font color='orange'>  Visualize Functions </font>

In [ ]:
def color_palette():
    """Color palette that maps each class to RGB values.

    This one is actually taken from ADE20k.
    """
    return [[0, 0, 0],  [92, 0, 255]]

palette = color_palette()

In [ ]:
def visualize_mask(image: np.ndarray, mask: np.ndarray):
    color_seg = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    palette = np.array(color_palette())
    
    for label, color in enumerate(palette):
        color_seg[mask == label, :] = color
    color_seg = color_seg[..., ::-1]  # convert to BGR

    img = np.array(image) * 0.5 + color_seg * 0.5  # plot the image with the segmentation map
    img = img.astype(np.uint8)

    return img

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

def visualize_seg_mask(test_dataset, predicted_segmentation_maps):
    row_num = 4
    col_num = 5
    step = 5

    fig = plt.figure(figsize=(8, 7))
    for row in range(0, 4):
        idx = np.random.randint(0, len(test_dataset))
        _,_,img, segmentation_map = test_dataset[idx]
        
        #img = batch["original_images"][idx]
        #segmentation_map = batch["original_segmentation_maps"][idx]
        pred_segmentation_map = predicted_segmentation_maps[idx].cpu().numpy()

        ax = fig.add_subplot(row_num, col_num, row*step+1, xticks=[], yticks=[])
        ax.imshow(img)
        ax.set_title('image')

        ax = fig.add_subplot(row_num, col_num, row*step+2, xticks=[], yticks=[])
        ax.imshow(segmentation_map)
        ax.set_title('ground truth')

        ax = fig.add_subplot(row_num, col_num, row*step+3, xticks=[], yticks=[])
        ax.imshow(pred_segmentation_map)
        ax.set_title('predict map')

        ax = fig.add_subplot(row_num, col_num, row*step+4, xticks=[], yticks=[])
        grdtruth_mask = visualize_mask(img, segmentation_map)
        ax.imshow(grdtruth_mask)
        ax.set_title('grd truth mask')

        ax = fig.add_subplot(row_num, col_num, row*step+5, xticks=[], yticks=[])
        img_mask = visualize_mask(img, pred_segmentation_map)
        ax.imshow(img_mask)
        ax.set_title('predict mask')

    plt.tight_layout()
    plt.show()

# <font color='orange'> Inference </font>

After training, we can use the model to make predictions on new data.
Let's showcase this one of the examples of a test batch.

In [ ]:
import copy

#for idx, (train_loader, valid_loader, test_loader) in enumerate(dataloaders): 
def train_fold_pipeline(idx, model, id2label, device, optimizer, metric, epochs=2):
    print('Training Fold '+ str(idx))
    train_loader, valid_loader, test_loader = dataloaders[idx]
    
    #train the model
    train_iou = train_one_fold(model, train_loader, valid_loader, 
          id2label, device, optimizer, metric, idx_fold=idx, epochs=epochs)
    print('train mean IoU: '+ str(train_iou))
    
    #load best model
    test_model = copy.deepcopy(model)
    test_model.load_state_dict(torch.load(f"/kaggle/working/fold{idx}_model.pt"))
    test_model = test_model.to(device)
    
    #predict segmentation map
    gnd_truth_maps, pred_maps = test_result(test_model, test_loader, id2label, device)
    metric.add_batch(references=gnd_truth_maps, predictions=pred_maps)
    print("Mean IoU:", metric.compute(ignore_index=-1, num_labels = len(id2label))['mean_iou'])
    
    return pred_maps

In [ ]:
##model = Mask2FormerForUniversalSegmentation.from_pretrained(
##    "facebook/mask2former-swin-base-coco-panoptic").to(device)

#predicted_segmentation_maps = train_fold_pipeline(
#    0, model, id2label, device, optimizer, metric, epochs=2)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[0]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
##model = Mask2FormerForUniversalSegmentation.from_pretrained(
##    "facebook/mask2former-swin-base-coco-panoptic").to(device)

#model.eval()
#predicted_segmentation_maps = train_fold_pipeline(
#    1, model, id2label, device, optimizer, metric, epochs=2)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[1]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
#predicted_segmentation_maps = train_fold_pipeline(
#    2, model, id2label, device, optimizer, metric, epochs=2)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[2]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
#predicted_segmentation_maps = train_fold_pipeline(
#    3, model, id2label, device, optimizer, metric, epochs=2)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[3]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
#model.eval()
#predicted_segmentation_maps = train_fold_pipeline(
#    4, model, id2label, device, optimizer, metric, epochs=2)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[4]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

# <font color='orange'> Train all class of dataset </font>

In [ ]:
datasets = []

for i in range(0,5):
    df_train = pd.read_csv(f'/kaggle/input/ebhi-seg-csv/all/train_fold_{i}.csv')
    df_test = pd.read_csv(f'/kaggle/input/ebhi-seg-csv/all/test_fold_{i}.csv')
    
    #df_train['img_path'] = df_train['image_path'].apply(lambda x: os.path.dirname(x))
    #df_train['lbl_path'] = df_train['image_path'].apply(
    #    lambda x: os.path.dirname(x).replace('image', 'label'))
    #df_train['image'] = df_train['image_path'].apply(lambda x: os.path.basename(x))
    #df_train = df_train[['img_path', 'lbl_path', 'image', 'label']]
    
    #df_test['img_path'] = df_test['image_path'].apply(lambda x: os.path.dirname(x))
    #df_test['lbl_path'] = df_test['image_path'].apply(
    #    lambda x: os.path.dirname(x).replace('image', 'label'))
    #df_test['image'] = df_test['image_path'].apply(lambda x: os.path.basename(x))
    #df_test = df_test[['img_path', 'lbl_path', 'image', 'label']]

    ## train test split
    dataset = ImageSegmentationDataset(df_train, data_path='/kaggle/input/ebhi-seg/', transform=train_transform)
    train_dataset, valid_dataset = train_val_split(dataset, validation_split=0.5)
    test_dataset = ImageSegmentationDataset(df_test, data_path='/kaggle/input/ebhi-seg/', transform=test_transform)
    
    datasets.append((train_dataset, valid_dataset, test_dataset))

In [ ]:
dataloaders = []
for i, (train_set, valid_set, test_set) in enumerate(datasets):
    train_dataloader = DataLoader(train_set, batch_size=8, shuffle=True, collate_fn=collate_fn)
    val_dataloader = DataLoader(valid_set, batch_size=4, shuffle=True, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_set, batch_size=4, shuffle=False, collate_fn=collate_fn)
    
    dataloaders.append((train_dataloader, val_dataloader, test_dataloader))

In [ ]:
import torch
train_dataloader = dataloaders[0][1]

batch = next(iter(train_dataloader))
for k,v in batch.items():
    print(type(v))
    if isinstance(v, torch.Tensor):
        print(k,v.shape)
    else:
        print(k,v[0].shape)

In [ ]:
#from transformers import Mask2FormerForUniversalSegmentation

#model = Mask2FormerForUniversalSegmentation.from_pretrained(
#    "facebook/mask2former-swin-base-coco-panoptic").to(device)

In [ ]:
model = PolypSegmentationNetwork().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2, factor=0.5)

In [ ]:
#predicted_segmentation_maps = train_fold_pipeline(
#    0, model, id2label, device, optimizer, metric, epochs=10)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[0]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[0]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
#predicted_segmentation_maps = train_fold_pipeline(
#    1, model, id2label, device, optimizer, metric, epochs=10)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[1]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[1]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
model.eval()

predicted_segmentation_maps = train_fold_pipeline(
    2, model, id2label, device, optimizer, metric, epochs=100)

In [ ]:
#visualize random images
_, _, test_dataset = datasets[2]
visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
#model.eval()

#predicted_segmentation_maps = train_fold_pipeline(
#    3, model, id2label, device, optimizer, metric, epochs=40)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[3]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[3]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
#model.eval()

#predicted_segmentation_maps = train_fold_pipeline(
#    4, model, id2label, device, optimizer, metric, epochs=8)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[4]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)

In [ ]:
##visualize random images
#_, _, test_dataset = datasets[4]
#visualize_seg_mask(test_dataset, predicted_segmentation_maps)